In [1]:

from numpy import core
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

#Pandas settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# seaborn settings
sns.set_style("darkgrid")


In [2]:
# Read in dataframe
chips_data = pd.read_csv('../data/chips_data.csv')
chips_data.head()


,Unnamed: 0,DATE,STORE_NBR,LYLTY_CARD_NBR,TXN_ID,PROD_NBR,PROD_NAME,PROD_QTY,TOT_SALES,LIFESTAGE,PREMIUM_CUSTOMER,PACK_WEIGHT,BRAND_NAME,AVG_CHIP_PRICE
0,0,2018-10-17,1,1000,1,5,Natural Chip Compny SeaSalt175g,2,6.0,YOUNG SINGLES/COUPLES,Premium,175,Natural Chip Co,3.00
1,1,2019-05-14,1,1307,348,66,CCs Nacho Cheese 175g,3,6.3,MIDAGE SINGLES/COUPLES,Budget,175,CCs,2.10
2,2,2019-05-20,1,1343,383,61,Smiths Crinkle Cut Chips Chicken 170g,2,2.9,MIDAGE SINGLES/COUPLES,Budget,170,Smiths,1.45
3,3,2018-08-17,2,2373,974,69,Smiths Chip Thinly S/Cream&Onion 175g,5,15.0,MIDAGE SINGLES/COUPLES,Budget,175,Smiths,3.00
4,4,2018-08-18,2,2426,1038,108,Kettle Tortilla ChpsHny&Jlpno Chili 150g,3,13.8,MIDAGE SINGLES/COUPLES,Budget,150,Kettle,4.60


In [3]:
# Checking that all our data in encoded correctly
chips_data.dtypes


Unnamed: 0            int64
DATE                 object
STORE_NBR             int64
LYLTY_CARD_NBR        int64
TXN_ID                int64
PROD_NBR              int64
PROD_NAME            object
PROD_QTY              int64
TOT_SALES           float64
LIFESTAGE            object
PREMIUM_CUSTOMER     object
PACK_WEIGHT           int64
BRAND_NAME           object
AVG_CHIP_PRICE      float64
dtype: object

In [4]:
# Date is encoded as object - change to datetime
chips_data['DATE'] = pd.to_datetime(chips_data['DATE'])


In [5]:
 # Create column YEAR_MONTH to store year and month from the date
chips_data['YEAR_MONTH'] = chips_data['DATE'].dt.strftime('%Y%m').astype('int')
chips_data.head()


,Unnamed: 0,DATE,STORE_NBR,LYLTY_CARD_NBR,TXN_ID,PROD_NBR,PROD_NAME,PROD_QTY,TOT_SALES,LIFESTAGE,PREMIUM_CUSTOMER,PACK_WEIGHT,BRAND_NAME,AVG_CHIP_PRICE,YEAR_MONTH
0,0,2018-10-17,1,1000,1,5,Natural Chip Compny SeaSalt175g,2,6.0,YOUNG SINGLES/COUPLES,Premium,175,Natural Chip Co,3.00,201810
1,1,2019-05-14,1,1307,348,66,CCs Nacho Cheese 175g,3,6.3,MIDAGE SINGLES/COUPLES,Budget,175,CCs,2.10,201905
2,2,2019-05-20,1,1343,383,61,Smiths Crinkle Cut Chips Chicken 170g,2,2.9,MIDAGE SINGLES/COUPLES,Budget,170,Smiths,1.45,201905
3,3,2018-08-17,2,2373,974,69,Smiths Chip Thinly S/Cream&Onion 175g,5,15.0,MIDAGE SINGLES/COUPLES,Budget,175,Smiths,3.00,201808
4,4,2018-08-18,2,2426,1038,108,Kettle Tortilla ChpsHny&Jlpno Chili 150g,3,13.8,MIDAGE SINGLES/COUPLES,Budget,150,Kettle,4.60,201808


In [6]:
# Task asks us to select control stores
# Trial stores are store numbers: 77, 86 and 88
# Hence we need stores that match these trial stores in terms of:

#           - Same sales
#           - Same num of customers
#           - Same transaction numbers

# Group stores by STORE_NBR and YEAR_MONTH - showing:
#           - sum of sales - $
#           - unique loyal card numbers - num of customers
#           - unique transaction id - num of transactions
#           - sum of product qty - product sold

store_selection = chips_data.groupby(['STORE_NBR','YEAR_MONTH']).agg(
                {'TOT_SALES': 'sum', 'LYLTY_CARD_NBR': 'nunique',
                 'TXN_ID': 'nunique', 'PROD_QTY': 'sum'})

# Create UNIT_PRICE column for unit price of chips
store_selection['AVG_UNIT_PRICE'] = (store_selection['TOT_SALES']/
                                store_selection['PROD_QTY'])

# Transactions per customer
store_selection['TXN_PER_LOYALTY_CARD_NUM'] = (store_selection['TXN_ID']/
                                            store_selection['LYLTY_CARD_NBR'])

# Reset index
store_selection = store_selection.reset_index()
store_selection.head()


,STORE_NBR,YEAR_MONTH,TOT_SALES,LYLTY_CARD_NBR,TXN_ID,PROD_QTY,AVG_UNIT_PRICE,TXN_PER_LOYALTY_CARD_NUM
0,1,201807,188.9,47,49,58,3.256897,1.042553
1,1,201808,168.4,41,41,52,3.238462,1.000000
2,1,201809,268.1,57,59,71,3.776056,1.035088
3,1,201810,175.4,39,40,51,3.439216,1.025641
4,1,201811,184.8,44,45,55,3.360000,1.022727


In [7]:
# Trial period start of Feb 19 - end of April 19
# Count num of times a store num appears (ie how many months of data)
stores_12_months = store_selection['STORE_NBR'].value_counts()
# Filter by stores with 12 months of data - store as index
stores_12_months = stores_12_months[stores_12_months == 12].index
# Filter index stores to store_selection df and map by store number
stores_12_months = (store_selection[store_selection['STORE_NBR']
                    .isin(stores_12_months)])

stores_12_months.info() # Stores with at least 12 months of data

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3108 entries, 0 to 3164
Data columns (total 8 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   STORE_NBR                 3108 non-null   int64  
 1   YEAR_MONTH                3108 non-null   int64  
 2   TOT_SALES                 3108 non-null   float64
 3   LYLTY_CARD_NBR            3108 non-null   int64  
 4   TXN_ID                    3108 non-null   int64  
 5   PROD_QTY                  3108 non-null   int64  
 6   AVG_UNIT_PRICE            3108 non-null   float64
 7   TXN_PER_LOYALTY_CARD_NUM  3108 non-null   float64
dtypes: float64(3), int64(5)
memory usage: 218.5 KB


In [8]:
# Correlation between trial and control stores based on chip sales

# Define function to calculate correlation
def storeCorr(metric_columns, trial_store_num, stores_12_months=stores_12_months):

        """ Function calculates correction between stores

        Args:
            metric_columns ([list]): [columns to compare]
            trial_store_num ([int]): [store number of trial store]
            stores_12_months ([df], optional): [df of stores with 12 months of sales]. Defaults to stores_12_months.

        Returns:
            [correlation]: [df of the correlation between the trial store and
                                control stores]
        """

# Extract control stores by taking inverse of the df where trial stores are (by store num)
        control_store_num = (stores_12_months[~stores_12_months['STORE_NBR'].isin([77,86.88])]
                        ['STORE_NBR'].unique())
# Create df with desired column names to store correlation
        correlation = pd.DataFrame(columns=['YEAR_MONTH', 'TRIAL_STORE',
                                         'CONTROL_STORE', 'CORR'])
# Extract the trial stores from the input table
        trial_stores = stores_12_months[stores_12_months['STORE_NBR']==
                                 trial_store_num][metric_columns].reset_index()
# Loop over the control stores
        for store in control_store_num:
                df = pd.DataFrame(columns= ['YEAR_MONTH', 'TRIAL_STORE', 'CONTROL_STORE'
                                    'CORR'])
# For each control store num extract all rows for that particular store
        control_store = stores_12_months[stores_12_months['STORE_NBR']== store][metric_columns].reset_index()
# Assign CORR column to the correlation with each trial store to new df
# (a row for each month and hence correlation for that month)
        df['CORR'] = trial_stores.corrwith(control_store, axis=1)
# Assign the trial store num to new df
        df['TRIAL_STORE'] = trial_store_num
# Assign the control store num to the df (a row for each month and hence correlation for that month)
        df['CONTROL_STORE'] = store
# Assign the year and month for the correlation based off the current input table row
        df['YEAR_MONTH'] = list(stores_12_months[stores_12_months['STORE_NBR']== trial_store_num]['YEAR_MONTH'])
# Combine the new df to our master correlation df 
        correlation = pd.concat([correlation, df])
        return correlation


In [9]:
# Create blank df for correlation
correlation_table = pd.DataFrame()
# Loop over each trial store
for trial_store in [77,86,88]:
    # For each trial store combine our df with the storeCorr function on selected columns
    correlation_table = pd.concat([correlation_table, storeCorr(['TOT_SALES',
                                                         'LYLTY_CARD_NBR',
                                                         'TXN_ID',
                                                         'TXN_PER_LOYALTY_CARD_NUM',
                                                         'AVG_UNIT_PRICE'], trial_store)])
correlation_table.head()
                                                    

,YEAR_MONTH,TRIAL_STORE,CONTROL_STORE,CORR,CONTROL_STORECORR
0,201807,77,272,0.984638,NaN
1,201808,77,272,0.985306,NaN
2,201809,77,272,0.989293,NaN
3,201810,77,272,0.996203,NaN
4,201811,77,272,0.990701,NaN


In [10]:
# Create function to compute magnitude distance
def store_magnitude (metric_columns, trial_store_num, stores_12_months=stores_12_months):
        """ Function calculates magnitude between stores

        Args:
            metric_columns ([list]): [columns to compare]
            trial_store_num ([int]): [store number of trial store]
            stores_12_months ([df], optional): [df of stores with 12 months of sales]. Defaults to stores_12_months.

        Returns: 
            [magnitude]: [df of the magnitude between the trial store and
                                control stores]
        """
# Extract control stores by taking inverse of the df where trial stores are (by store num)
        control_store_num = (stores_12_months[~stores_12_months['STORE_NBR']
                                .isin([77,86,88])]['STORE_NBR'].unique())
# Initialize empty df
        magnitude = pd.DataFrame()
# Extract the trial stores from the input table
        trial_stores = stores_12_months[stores_12_months['STORE_NBR']== trial_store_num][metric_columns]
# Loop over the control stores
        for store in control_store_num:
# Take absolute value of: (trial store metric columns of interest minus (-) the control store metric columns 
                df = abs(stores_12_months[stores_12_months['STORE_NBR']== trial_store_num].reset_index()[metric_columns]
                        - stores_12_months[stores_12_months['STORE_NBR']==store].reset_index()[metric_columns])
# Assign the trial store num to new df
                df['TRIAL_STORE'] = trial_store_num
# Assign the control store num to the df (a row for each month and hence magnitude for that month)
                df['CONTROL_STORE'] = store
# Assign the year and month for the magnitude based off the current input table row
                df['YEAR_MONTH'] = list(stores_12_months[stores_12_months['STORE_NBR']== trial_store_num]['YEAR_MONTH'])
# Concat the two df together
                magnitude = pd.concat([magnitude, df])
# Loop over each column of interest
        for column in metric_columns:
# Compute the magnitude:
# 1- (Observed distance – minimum distance)/(Maximum distance – minimum distance)
                magnitude[column] = 1 - ((magnitude[column] - magnitude[column].min()) /
                                        (magnitude[column].max() - magnitude[column].min()))
# Assign the mean value of the metric_columns for each store to a new column called magnitude
        magnitude['MAGNITUDE'] = magnitude[metric_columns].mean(axis=1)
# Return the df
        return magnitude


In [11]:
# Create blank df for magnitude
magnitude = pd.DataFrame()
# Loop over each trial store
for trial_store in [77,86,88]:
    # For each trial store combine our df with the store_magnitude function on selected columns
    magnitude = pd.concat([magnitude, store_magnitude(['TOT_SALES',
                                                         'LYLTY_CARD_NBR',
                                                         'TXN_ID',
                                                         'TXN_PER_LOYALTY_CARD_NUM',
                                                         'AVG_UNIT_PRICE'], trial_store)])
magnitude.head()


,TOT_SALES,LYLTY_CARD_NBR,TXN_ID,TXN_PER_LOYALTY_CARD_NUM,AVG_UNIT_PRICE,TRIAL_STORE,CONTROL_STORE,YEAR_MONTH,MAGNITUDE
0,0.939141,1.000000,0.985185,0.910742,0.870640,77,1,201807,0.941142
1,0.939447,0.950495,0.962963,1.000000,0.860104,77,1,201808,0.942602
2,0.960729,0.831683,0.874074,0.968721,0.722300,77,1,201809,0.871501
3,0.985532,0.970297,0.977778,0.995518,0.747415,77,1,201810,0.935308
4,0.969303,0.950495,0.977778,0.886321,0.881368,77,1,201811,0.933053


In [12]:
# Combine both the correlation and magnitude columns
def combine_mag_corr(metric_columns, trial_store_num, stores_12_months=stores_12_months):
        """ Function combines magnitude and correlation between stores

        Args:
            metric_columns ([list]): [columns to compare]
            trial_store_num ([int]): [store number of trial store]
            stores_12_months ([df], optional): [df of stores with 12 months of sales]. Defaults to stores_12_months.

        Returns: 
            [master]: [df of both the magnitude and correlation between the trial store and
                                control stores]
        """
        # Compute correlation with storeCorr function
        correlation = storeCorr(metric_columns, trial_store_num, stores_12_months)
        # Compute magnitude with store_magnitude function
        magnitude = store_magnitude(metric_columns, trial_store_num, stores_12_months)
        # Drop 
        magnitude = magnitude.drop(metric_columns, axis=1)
        master = pd.merge(correlation, magnitude, on=['CONTROL_STORE', 'TRIAL_STORE', 'YEAR_MONTH'])
        return master


In [13]:
# Compare stores based off total sales
master_sales = pd.DataFrame()
# Loop over trial stores
for trial_store in [77, 86, 88]:
        # Call combine_mag_corr
        master_sales = pd.concat([master_sales, combine_mag_corr(['TOT_SALES'], trial_store)])


In [14]:
# Group sales by trial and control stores and take mean
sales_compare = master_sales.groupby(['TRIAL_STORE', 'CONTROL_STORE']).mean().reset_index()
# Combine correlation and magnitude to obtain a new scoring method (0.5 weight for magnitude and 0.5 for correlation )
# This effectively takes each measure into equal consideration for analysis
sales_compare['master_score'] = (0.5*sales_compare['CORR']) + (0.5*sales_compare['MAGNITUDE'])
# For each trial store print the top store with the highest magnitude for total sales
for trial_store in [77, 86, 88]:
        print(sales_compare[sales_compare['TRIAL_STORE']==trial_store].sort_values(ascending=False, by='master_score'))


   TRIAL_STORE  CONTROL_STORE  CORR  MAGNITUDE  master_score
0           77            272   1.0   0.900498      0.950249
   TRIAL_STORE  CONTROL_STORE  CORR  MAGNITUDE  master_score
1           86            272   1.0    0.50096       0.75048
   TRIAL_STORE  CONTROL_STORE  CORR  MAGNITUDE  master_score
2           88            272  -1.0   0.368907     -0.315546


In [15]:
# Compare stores based off number of customers (loyalty card info)
master_customers = pd.DataFrame()
# Loop over trial stores
for trial_store in [77, 86, 88]:
        # Call combine_mag_corr
        master_customers = pd.concat([master_customers, combine_mag_corr(['LYLTY_CARD_NBR'], trial_store)])


In [16]:
customer_compare = master_customers.groupby(['TRIAL_STORE', 'CONTROL_STORE']).mean().reset_index()
# Combine correlation and magnitude to obtain a new scoring method (0.5 weight for magnitude and 0.5 for correlation )
# This effectively takes each measure into equal consideration for analysis
customer_compare['master_score'] = (0.5*customer_compare['CORR']) + (0.5*customer_compare['MAGNITUDE'])
# For each trial store print the top store with the highest magnitude for total sales
for trial_store in [77, 86, 88]:
        print(customer_compare[customer_compare['TRIAL_STORE']==trial_store].sort_values(ascending=False, by='master_score'))


   TRIAL_STORE  CONTROL_STORE  CORR  MAGNITUDE  master_score
0           77            272   1.0   0.941419       0.97071
   TRIAL_STORE  CONTROL_STORE  CORR  MAGNITUDE  master_score
1           86            272   1.0   0.470126      0.735063
   TRIAL_STORE  CONTROL_STORE  CORR  MAGNITUDE  master_score
2           88            272   1.0   0.388677      0.694338


In [17]:
# Loop over each trial store
for trial_store in [77, 86, 88]:
        # Set x to be where our sales match our trial stores the best by master score
        x = (sales_compare[sales_compare['TRIAL_STORE']== trial_store].sort_values(ascending=False, by='master_score')
                .set_index(['TRIAL_STORE', 'CONTROL_STORE'])['master_score'])

        # Set y to be where our customer numbers match our trial stores the most by master score
        y = (customer_compare[customer_compare['TRIAL_STORE']== trial_store].sort_values(ascending=False,
                 by='master_score').set_index(['TRIAL_STORE', 'CONTROL_STORE'])['master_score'])

        print((pd.concat([x,y],axis=1).sum(axis=1)/2).sort_values(ascending=False).head(3))



TRIAL_STORE  CONTROL_STORE
77           272              0.960479
dtype: float64
TRIAL_STORE  CONTROL_STORE
86           272              0.742772
dtype: float64
TRIAL_STORE  CONTROL_STORE
88           272              0.189396
dtype: float64
